In [10]:
import pandas as pd
import os

# Importeer lokale modules
from preprocessing import (
    transformeer_woonderzoek_limburg,
    transformeer_woonderzoek_nederland,
    transformeer_woonderzoek_data,
    transformeer_leefbarometer_data,
    laad_woningtekort_data,
    laad_data_invoerapplicatie
)

In [11]:
# -----------------------------------------------------------------------------
# Instellingen en mappings
# -----------------------------------------------------------------------------
# Mapping van regio's
# Dit kan later verbeterd worden door alle buurt-wijk-gem-corop-prov mappings centraal op te slaan

regio_mapping = {
    "Nederland": "nl00",
    "Noord-Limburg": "cr37",
    "Midden-Limburg": "cr38",
    "Zuid-Limburg": "cr39",
}

In [12]:
# -----------------------------------------------------------------------------
# Initialiseer de indicator dictionary
# -----------------------------------------------------------------------------
# Hierin worden indicatornamen gekoppeld aan hun corresponderende DataFrames

indicatoren_dict = {}

In [13]:
# -----------------------------------------------------------------------------
# Indicator MO_11a: Woningtekort Data
# -----------------------------------------------------------------------------
print("Processing indicator: MO_11a")
df_mo_11a = laad_woningtekort_data(regio_mapping)

# Toevoegen aan de indicator dictionary
indicatoren_dict["MO_11a"] = df_mo_11a[["geoitem", "geolevel", "aantal", "period"]]

Processing indicator: MO_11a


c:\Users\damoiseauxjmb\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\damoiseauxjmb\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [14]:
# -----------------------------------------------------------------------------
# Indicator MO_11b: Woononderzoek Nederland en Limburg
# -----------------------------------------------------------------------------
print("Processing indicator: MO_11b")
df_woonderzoek_limburg = pd.read_excel("../../../data/Woononderzoek_nederland/Tevreden met woning - Corop-gebieden.xlsx")
df_woonderzoek_nederland = pd.read_excel("../../../data/Woononderzoek_nederland/Tevreden met woning - Nederland.xlsx")

# Preprocessing
df_woonderzoek_limburg_pp = transformeer_woonderzoek_limburg(df=df_woonderzoek_limburg, n_rows=5)
df_woonderzoek_nederland_pp = transformeer_woonderzoek_nederland(df=df_woonderzoek_nederland, n_rows=5)

# Column renames specifiek voor MO_11b
column_renames_mo_11b = {
    "Waarde": "MO_11b",
    "Categorie": "dim_tevredenheid_0",
}

# Combineer en verwerk
df_mo_11b = pd.concat([df_woonderzoek_limburg_pp, df_woonderzoek_nederland_pp])
df_mo_11b = transformeer_woonderzoek_data(df_mo_11b, regio_mapping, column_renames_mo_11b)

# Toevoegen aan de indicator dictionary
indicatoren_dict["MO_11b"] = df_mo_11b[["geoitem", "geolevel", "MO_11b", "dim_tevredenheid_0", "period"]]


Processing indicator: MO_11b


c:\Users\damoiseauxjmb\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\damoiseauxjmb\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [15]:
# -----------------------------------------------------------------------------
# Indicator MO_10a, D_39a, D_39aa: Leefbarometer Data
# -----------------------------------------------------------------------------
print("Processing indicators: MO_10a, D_39a, D_39aa")
relevante_jaren = [2014, 2018, 2020, 2022]
column_renames = {
    "lbm": "MO_10a",
    "fys": "D_39a",
    "vrz": "D_39aa",
    "COROP_NAAM": "geoitem",
    "jaar": "period",
}

# Data verwerken
df_leefbarometer = transformeer_leefbarometer_data(
    input_bestand_pad="../../../data/Leefbarometer/Leefbaarometer-scores buurten 2002-2022.csv",
    bu_code_correcties_pad="../../../data/Buurtcodes/bu_code_correcties.xlsx",
    limburg_buurten_pad="../../../data/Buurtcodes/BU_WK_GM_codes.xls",
    relevante_jaren=relevante_jaren,
    column_renames=column_renames,
    regio_mapping=regio_mapping,
)

# Los toevoegen van indicatoren aan de dictionary
for indicator_code in ["MO_10a", "D_39a", "D_39aa"]:
    indicatoren_dict[indicator_code] = df_leefbarometer[["geoitem", "geolevel", "period", indicator_code]]

Processing indicators: MO_10a, D_39a, D_39aa


In [16]:
# -----------------------------------------------------------------------------
# Indicators R_118a, R_119a, R_120a, R_121a, R_122a, R_124a: Data Invoerapplicatie
# -----------------------------------------------------------------------------
print("Processing indicators: R_118a, R_119a, R_120a, R_121a, R_122a, R_124a")
df_invoerapplicatie = laad_data_invoerapplicatie(
    bron_bestand="../../../data/Invoerapplicatie/Invoerapplicatie Leefbare steden en dorpen.xlsx"
)

# Los toevoegen van indicatoren aan de dictionary
for indicator_code in ["R_118a", "R_119a", "R_120a", "R_121a", "R_122a", "R_124a"]:
    indicatoren_dict[indicator_code] = df_invoerapplicatie[["geoitem", "geolevel", indicator_code]]


Processing indicators: R_118a, R_119a, R_120a, R_121a, R_122a, R_124a


In [ ]:
# -----------------------------------------------------------------------------
# Controleer de inhoud van de dictionary
# -----------------------------------------------------------------------------
print("\nControle van indicator dictionary:")
for key, value in indicatoren_dict.items():
    if isinstance(value, pd.DataFrame):
        print(f"Indicator {key}, bevat een DataFrame met shape: {value.shape}")
    else:
        print(f"Waarschuwing: Indicator {key} bevat geen geldig DataFrame")


Controle van indicator dictionary:
Indicator: MO_11a, bevat een DataFrame met shape: (21, 4)
Indicator: MO_11b, bevat een DataFrame met shape: (95, 5)
Indicator: MO_10a, bevat een DataFrame met shape: (12, 4)
Indicator: D_39a, bevat een DataFrame met shape: (12, 4)
Indicator: D_39aa, bevat een DataFrame met shape: (12, 4)
Indicator: R_118a, bevat een DataFrame met shape: (1, 3)
Indicator: R_119a, bevat een DataFrame met shape: (1, 3)
Indicator: R_120a, bevat een DataFrame met shape: (1, 3)
Indicator: R_121a, bevat een DataFrame met shape: (1, 3)
Indicator: R_122a, bevat een DataFrame met shape: (1, 3)
Indicator: R_124a, bevat een DataFrame met shape: (1, 3)


In [18]:
# -----------------------------------------------------------------------------
# Schrijf alle DataFrames weg naar de output folder
# -----------------------------------------------------------------------------
print("\nData wegschrijven naar bestanden...")
output_folder = "../../../Publicatie_bestanden/Leefbare_steden_en_dorpen"
for indicator, df in indicatoren_dict.items():
    bestandspad = os.path.join(output_folder, f"{indicator}.csv")
    df.to_csv(bestandspad, index=False)
    print(f"{indicator} opgeslagen als: {bestandspad}")


Data wegschrijven naar bestanden...
MO_11a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\MO_11a.csv
MO_11b opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\MO_11b.csv
MO_10a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\MO_10a.csv
D_39a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\D_39a.csv
D_39aa opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\D_39aa.csv
R_118a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\R_118a.csv
R_119a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\R_119a.csv
R_120a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\R_120a.csv
R_121a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\R_121a.csv
R_122a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\R_122a.csv
R_124a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_ste